In [2]:
import sqlite3
import pandas as pd

In [3]:
con = sqlite3.connect(':memory:')

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [4]:
data = pd.read_csv('boston.csv')
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [5]:
data.to_sql('boston', con, index=False, if_exists='replace')

506

In [6]:
pd.read_sql(
    '''
    SELECT * FROM boston
    limit 5
    ''',
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


### Все задания нужно выполнить только с помощью sql

1. Выведите количество пустых значений по колонкам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во пустых значений)

In [7]:
pd.read_sql(
    '''
    SELECT DISTINCT COUNT(*) FILTER(WHERE CRIM IS NULL) OVER () as 'CRIM'
          ,COUNT(*) FILTER(WHERE ZN IS NULL) OVER () as 'ZN'
          ,COUNT(*) FILTER(WHERE INDUS IS NULL) OVER () as 'INDUS'
          ,COUNT(*) FILTER(WHERE CHAS IS NULL) OVER () as 'CHAS'
          ,COUNT(*) FILTER(WHERE NOX IS NULL) OVER () as 'NOX'
    FROM boston
    ''',
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX
0,0,0,0,0,0


2. Выведите количество уникальных значений по колонокам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во уникальных значений)

In [8]:
pd.read_sql(
    '''
    SELECT MAX(CRIM_RANK) as CRIM
          ,MAX(ZN_RANK) as 'ZN'
          ,MAX(INDUS_RANK) as 'INDUS'
          ,MAX(CHAS_RANK) as 'CHAS'
          ,MAX(NOX_RANK) as 'NOX'
    FROM (
                    SELECT *
                        ,DENSE_RANK() OVER (ORDER BY CRIM) as 'CRIM_RANK'
                        ,DENSE_RANK() OVER (ORDER BY ZN) as 'ZN_RANK'
                        ,DENSE_RANK() OVER (ORDER BY INDUS) as 'INDUS_RANK'
                        ,DENSE_RANK() OVER (ORDER BY CHAS) as 'CHAS_RANK'
                        ,DENSE_RANK() OVER (ORDER BY NOX) as 'NOX_RANK'
                    FROM boston
          ) t1  
    ''',
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX
0,504,26,76,2,81


3. Выведите колонки, у которых медиана равна минимальному значению (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX.
Напишите какой вывод можно сделать по данным в этих колонках

In [29]:
# если минимум равен медиане, то как минимум половина значений должна быть равна минимальному, 
# другими словами, максимальное значение во втором квартиле должно быть равно минимальному 
# значению в первом квартиле. Решение получилось громоздким, но ничего лучше не придумал, чтобы 
# вывести именно названия колонок
# Выводы: по параметрам CHAS и ZN более половины значений равны нулю, т.е. в более, чем половине наблюдений
# нет границ с рекой; в более, чем половине наблюдений (не обязательно совпадают с CHAS) доля земель под жилую 
# застройку не зонирована под участки площадью более 25 000 кв. футов
pd.read_sql(
    '''
    SELECT Parameter
    FROM (
            SELECT 'CRIM' as 'Parameter'
                ,MIN(CRIM) as 'MIN'
                ,MAX(CRIM) as 'MEDIAN'
            FROM  (
                SELECT CRIM
                        ,NTILE(2) OVER (ORDER BY CRIM) as 'Quartiles'
                FROM boston
                ) t1
            WHERE 1=1
                AND Quartiles = 1
            UNION
            SELECT 'ZN' as 'Parameter'
                ,MIN(ZN) as 'MIN'
                ,MAX(ZN) as 'MEDIAN'
            FROM  (
                SELECT ZN
                        ,NTILE(2) OVER (ORDER BY ZN) as 'Quartiles'
                FROM boston
                ) t2
            WHERE 1=1
                AND Quartiles = 1
            UNION
            SELECT 'INDUS' as 'Parameter'
                ,MIN(INDUS) as 'MIN'
                ,MAX(INDUS) as 'MEDIAN'
            FROM  (
                SELECT INDUS
                        ,NTILE(2) OVER (ORDER BY INDUS) as 'Quartiles'
                FROM boston
                ) t3
            WHERE 1=1
                AND Quartiles = 1
            UNION
            SELECT 'CHAS' as 'Parameter'
                ,MIN(CHAS) as 'MIN'
                ,MAX(CHAS) as 'MEDIAN'
            FROM  (
                SELECT CHAS
                        ,NTILE(2) OVER (ORDER BY CHAS) as 'Quartiles'
                FROM boston
                ) t4
            WHERE 1=1
                AND Quartiles = 1
            UNION
            SELECT 'NOX' as 'Parameter'
                ,MIN(NOX) as 'MIN'
                ,MAX(NOX) as 'MEDIAN'
            FROM  (
                SELECT NOX
                        ,NTILE(2) OVER (ORDER BY NOX) as 'Quartiles'
                FROM boston
                ) t5
            WHERE 1=1
                AND Quartiles = 1
                ) t5
    WHERE MIN = MEDIAN
    ''',
    con,
)

,Parameter
0,CHAS
1,ZN


4. Выведите ранги значений колонки LSTAT(процент населения с более низким статусом) в домах с самой дорогой стоимостью (значение LSTAT, стоимость, ранг) среди всех значений LSTAT. Напишите какой вывод можно сделать по этим данным.

In [46]:

pd.read_sql(
    '''
    SELECT LSTAT
          ,MEDV
          ,RANK() OVER (PARTITION BY MEDV ORDER BY LSTAT) as 'RANK'
    FROM boston
    WHERE 1=1
        AND MEDV = (SELECT MAX(MEDV) FROM boston)
    ''',
    con,
)

,LSTAT,MEDV,RANK
0,1.73,50.0,1
1,1.92,50.0,2
2,2.88,50.0,3
3,2.96,50.0,4
4,2.97,50.0,5
5,3.16,50.0,6
6,3.26,50.0,7
7,3.32,50.0,8
8,3.70,50.0,9
9,3.73,50.0,10


Процент более низкого статуса населения среди самой дорогой недвижимости не превышает 10%

5. Выведите среднюю стоимость домов граничащих с рекой(CHAS) и нет (граничит/не граничит, стоимость)

In [44]:

pd.read_sql(
    '''
    SELECT DISTINCT IIF(CHAS = 1, 'Граничит с рекой', 'Не граничит с рекой') as 'CHAS'
          ,AVG(MEDV) OVER (PARTITION BY CHAS) as 'MEDV' 
    FROM boston
    limit 5
    ''',
    con,
)

,CHAS,MEDV
0,Не граничит с рекой,22.093843
1,Граничит с рекой,28.440000


6. Выведите все колонки, у которых среднее значение выше, когда дом граничит с рекой (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX. Напишите какой вывод можно сделать по этим данным.

In [58]:

# выбирать будем по всем, кроме CHAS - среднее значение для граничащих с рекой будет 1 
# и 0 в противном случае
pd.read_sql(
    '''
    SELECT Parameter
    FROM (
            SELECT DISTINCT 'CRIM' as 'Parameter'
                ,CHAS
                ,AVG
                ,RANK() OVER (ORDER BY AVG DESC) as 'RANK'
            FROM (SELECT CHAS
                        ,AVG(CRIM) OVER (PARTITION BY CHAS) as 'AVG'
                FROM boston) t1
            UNION 
            SELECT DISTINCT 'ZN' as 'Parameter'
                ,CHAS
                ,AVG
                ,RANK() OVER (ORDER BY AVG DESC) as 'RANK'
            FROM (SELECT CHAS
                        ,AVG(ZN) OVER (PARTITION BY CHAS) as 'AVG'
                FROM boston) t2
            UNION
            SELECT DISTINCT 'INDUS' as 'Parameter'
                ,CHAS
                ,AVG
                ,RANK() OVER (ORDER BY AVG DESC) as 'RANK'
            FROM (SELECT CHAS
                        ,AVG(INDUS) OVER (PARTITION BY CHAS) as 'AVG'
                FROM boston) t3
            UNION
            SELECT DISTINCT 'NOX' as 'Parameter'
                ,CHAS
                ,AVG
                ,RANK() OVER (ORDER BY AVG DESC) as 'RANK'
            FROM (SELECT CHAS
                        ,AVG(NOX) OVER (PARTITION BY CHAS) as 'AVG'
                FROM boston) t4
            ) t5
    WHERE 1=1
        AND CHAS = RANK
    ''',
    con,
)

,Parameter
0,INDUS
1,NOX


Доля акров неторгового бизнеса на город  и концентрация оксидов азота в среднем больше в районах, которые граничат с рекой, в сравнении с райноами, которые не граничат с рекой

7. Выведите значения долей промышленной застройки(INDUS), концентрации оксидов азота(NOX) и по их перцентилям - 10, 20 ... 100 ( перцетиль(10,20...100),значение INDUS, значение NOX). Напишите прослеживается между ними взаимосвязь

In [63]:

pd.read_sql(
    '''
    SELECT Percentile
          ,AVG(INDUS) as 'Avg_INDUS'
          ,AVG(NOX) as 'Avg_NOX'
    FROM (
        SELECT NTILE(10) OVER (ORDER BY INDUS) 'Percentile'
            ,INDUS
            ,NOX
        FROM boston
        ) t1
    GROUP BY Percentile
    ''',
    con,
)

,Percentile,Avg_INDUS,Avg_NOX
0,1,2.016078,0.438378
1,2,3.645294,0.494129
2,3,5.246275,0.448235
3,4,6.471765,0.470839
4,5,8.256078,0.531510
5,6,10.749216,0.515176
6,7,17.245800,0.649660
7,8,18.100000,0.675520
8,9,18.307200,0.676480
9,10,21.938800,0.655620


Прослеживается взаимосвязь между INDUS и NOX, а именно: чем выше доля промышленной застройки, тем выше уровень концентрации оксидов азота